# NY Times Best Sellers

* [New York Times Best Sellers n- Hardcover Fiction Best Sellers from 2008 to 2018](https://www.kaggle.com/cmenca/new-york-times-hardcover-fiction-best-sellers)

* [PySpark and SparkSQL Basics](https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53)

In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [3]:
import os
import sys
import gc
from datetime import (
    datetime,
    date
)
import numpy as np
import pandas as pd

#  Environemnt Variables

## Hadoop

In [4]:
os.environ['HADOOP_CONF_DIR'] = "/opt/hadoop/hadoop-3.2.2/etc/hadoop"

In [5]:
%%bash
export HADOOP_CONF_DIR="/opt/hadoop/hadoop-3.2.2/etc/hadoop"
ls $HADOOP_CONF_DIR | head -n 5

capacity-scheduler.xml
configuration.xsl
container-executor.cfg
core-site.xml
core-site.xml.48132.2022-02-15@12:29:41~


## PYTHONPATH

Refer to the **pyspark** modules to load from the ```$SPARK_HOME/python/lib``` in the Spark installation.

* [PySpark Getting Started](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

> Ensure the SPARK_HOME environment variable points to the directory where the tar file has been extracted. Update PYTHONPATH environment variable such that it can find the PySpark and Py4J under SPARK_HOME/python/lib. One example of doing this is shown below:

```
export PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
```

Alternatively install **pyspark** with pip or conda locally which installs the Spark runtime libararies (for standalone).

* [Can PySpark work without Spark?](https://stackoverflow.com/questions/51728177/can-pyspark-work-without-spark)

> As of v2.2, executing pip install pyspark will install Spark. If you're going to use Pyspark it's clearly the simplest way to get started. On my system Spark is installed inside my virtual environment (miniconda) at lib/python3.6/site-packages/pyspark/jars  
> PySpark has a Spark installation installed. If installed through pip3, you can find it with pip3 show pyspark. Ex. for me it is at ~/.local/lib/python3.8/site-packages/pyspark. This is a standalone configuration so it can't be used for managing clusters like a full Spark installation.

In [6]:
# os.environ['PYTHONPATH'] = "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip:/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
sys.path.extend([
    "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip",
    "/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
])

## PySpark package imports

Execute after the PYTHONPATH setup.

In [67]:
import pyspark.sql 
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col,
    lit,
    avg,
    stddev,
    isnan,
    date_format,
    to_date,
    trunc,
    months_between,
    add_months,
    regexp_extract,
    regexp_replace,
    from_unixtime,
)

---
# Spark Session


In [8]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder\
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.debug.maxToStringFields', 100) \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()

2022-02-22 18:41:05,094 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-02-22 18:41:06,990 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-02-22 18:41:08,520 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [10]:
NUM_CORES = 4
NUM_PARTITIONS = 3

spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

# DataFrame from Python data

* [SparkSession.createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True)](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html)

# Schema Definition

* [Data Types](https://spark.apache.org/docs/latest/sql-ref-datatypes.html#data-types)

```from pyspark.sql.types import *```

| Data type | Value type in Python | API to access or create a data type |  |
|:---|:---|:---|:--|
|ByteType | int or long Note: Numbers will be converted to 1-byte signed integer numbers at runtime. Please make sure that numbers are within the range of -128 to 127. | ByteType() |  |
| ShortType | int or long Note: Numbers will be converted to 2-byte signed integer numbers at runtime. Please make sure that numbers are within the range of -32768 to 32767. | ShortType() |  |
| IntegerType | int or long | IntegerType() |  |
| LongType | long Note: Numbers will be converted to 8-byte signed integer numbers at runtime. Please make sure that numbers are within the range of -9223372036854775808 to 9223372036854775807.Otherwise, please convert data to decimal.Decimal and use DecimalType. | LongType() |  |
| FloatType | float Note: Numbers will be converted to 4-byte single-precision floating point numbers at runtime. | FloatType() |  |
| DoubleType | float | DoubleType() |  |
| DecimalType | decimal.Decimal | DecimalType() |  |
| StringType | string | StringType() |  |
| BinaryType | bytearray | BinaryType() |  |
| BooleanType | bool | BooleanType() |  |
| TimestampType | datetime.datetime | TimestampType() |  |
| DateType | datetime.date | DateType() |  |
| ArrayType | list, tuple, or array | ArrayType(elementType, [containsNull]) Note:The default value of containsNull is True. |  |
| MapType | dict | MapType(keyType, valueType, [valueContainsNull]) Note:The default value of valueContainsNull is True. |  |
| StructType | list or tuple | StructType(fields) Note: fields is a Seq of StructFields. Also, two fields with the same name are not allowed. |  |
| StructField | The value type in Python of the data type of this field (For example, Int for a StructField with the data type IntegerType) | StructField(name, dataType, [nullable]) Note: The default value of nullable is True. |  |


# Data


In [23]:
%%bash
cd ./data/books
unzip -o kaggle_nytimes_bestsellers.zip 

hdfs dfs -mkdir -p books
hdfs dfs -put -f nyt2.json books/

Archive:  kaggle_nytimes_bestsellers.zip
  inflating: nyt2.json               


In [74]:
preview = spark.read\
    .option("compression", "none")\
    .option("inferSchema", True)\
    .json("books/nyt2.json")

preview.printSchema()
del preview

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- amazon_product_url: string (nullable = true)
 |-- author: string (nullable = true)
 |-- bestsellers_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: struct (nullable = true)
 |    |-- $numberDouble: string (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- published_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- rank: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- rank_last_week: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- title: string (nullable = true)
 |-- weeks_on_list: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)



In [76]:
books = spark\
    .read\
    .option("compression", "none")\
    .option("inferSchema", True)\
    .json("books/nyt2.json")\
    .select(
        "title",
        "author",
        col("price.$numberDouble").alias("price"),    
        to_date(
            from_unixtime(col("published_date.$date.$numberLong") / 1000),
            "yyyy-MM-dd"
        ).alias("published"),
        to_date(
            from_unixtime(col("bestsellers_date.$date.$numberLong") / 1000),
            "yyyy-MM-dd"
        ).alias("best_seller_date"),
        col("amazon_product_url").alias("url")
    )
books.printSchema()
books.show(truncate=False)

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- price: string (nullable = true)
 |-- published: date (nullable = true)
 |-- best_seller_date: date (nullable = true)
 |-- url: string (nullable = true)

+--------------------------------------------------+----------------------------------------+-----+----------+----------------+--------------------------------------------------------------------------------------------------+
|title                                             |author                                  |price|published |best_seller_date|url                                                                                               |
+--------------------------------------------------+----------------------------------------+-----+----------+----------------+--------------------------------------------------------------------------------------------------+
|ODD HOURS                                         |Dean R Koontz                

In [103]:
books.where(col("author").like("Alan%")).show(3)

+-------------------+----------+-----+----------+----------------+--------------------+
|              title|    author|price| published|best_seller_date|                 url|
+-------------------+----------+-----+----------+----------------+--------------------+
|THE SPIES OF WARSAW|Alan Furst| null|2008-06-22|      2008-06-07|http://www.amazon...|
|THE SPIES OF WARSAW|Alan Furst| null|2008-06-29|      2008-06-14|http://www.amazon...|
|THE SPIES OF WARSAW|Alan Furst| null|2008-07-06|      2008-06-21|http://www.amazon...|
+-------------------+----------+-----+----------+----------------+--------------------+
only showing top 3 rows



In [105]:
books.where(col("author").startswith("Alan")).show(3)

+-------------------+----------+-----+----------+----------------+--------------------+
|              title|    author|price| published|best_seller_date|                 url|
+-------------------+----------+-----+----------+----------------+--------------------+
|THE SPIES OF WARSAW|Alan Furst| null|2008-06-22|      2008-06-07|http://www.amazon...|
|THE SPIES OF WARSAW|Alan Furst| null|2008-06-29|      2008-06-14|http://www.amazon...|
|THE SPIES OF WARSAW|Alan Furst| null|2008-07-06|      2008-06-21|http://www.amazon...|
+-------------------+----------+-----+----------+----------------+--------------------+
only showing top 3 rows



---
# Stop Spark Session

In [106]:
spark.stop()



# Cleanup

In [107]:
del spark
gc.collect()

7191